<a href="https://colab.research.google.com/github/carlangastr/marketing-science-projects/blob/main/Google_Cloud_Vision_%7C_API_%7C_Boosting_ads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction to Google Cloud Vision:**
----------
In the next post, I will show a usage approach to the Google Cloud vision API.

In the following example we will be using tables which contain supermetrics information and are stored in our SQL database.

Within the database we will have the unique URLs of each ad and we will use these URLs to analyze the context of the image stored in each one of them.

In [ ]:
!pip install webcolors google-cloud-vision

After installing the necessary additional libraries or packages, we need to import them.

In [ ]:
import os, io
from google.cloud import vision
import pandas as pd
from pandas import DataFrame

import webcolors

from PIL import Image as pilmage
import requests
from io import BytesIO

import json

import re
import ast

**Functions:**
----------------------
Cloud Vision will return information from the RGB color guide of our images when we request the color, therefore, we generate a function to classify the colors according to their closest primary color and obtain the name of the color and not its RGB code.

In [ ]:
#funtion to get the color

def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

#funtions to get the color

In [ ]:
#we are in a Google machine, we load our credentials in JSON format in our environment.

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'credentials.json'
client = vision.ImageAnnotatorClient()
image = vision.Image()

In [ ]:
#We create an empty dataframe and generate a query to our databases in bigqguery.

colors_dataframe = pd.DataFrame({'url': [], 'pixel_fraction':[], 'score':[], 'reed':[], 'gree':[], 'blue':[]})

query_url = """

SELECT 
source,   
ad_id,   
image_asset_url as url
FROM `main_url_dh`
where source = 'Facebook ads' and image_asset_url != '0'
group by 1,2,3  

UNION ALL 

SELECT 
source,   
ad_id,   
promoted_tweet_card_image_url as url
FROM `main_url_dh`
where source = 'Twitter ads' and promoted_tweet_card_image_url != '0'
group by 1,2,3  

"""
dataframe = pd.read_gbq(query_url)

In [ ]:
dataframe_dupli = dataframe.drop_duplicates(subset = ['url']).copy()
dataframe_dupli.reset_index(inplace = True)

#We certify that there are no duplicate records in our dataframe

In [ ]:
dataframe_dupli.url.shape

(1108,)

In [ ]:
#Our jsons, which are a response from Cloud Vision, contain a particular format, which we need to adjust before we can process them in our dataframe, 
#for this we create a function according to the type of information that we request from Cloud Vision.

def construction_of_json(string, model):
  
  if model == 'colors':
    b = string.replace('[', '').replace(']', '}').replace('color {\n', 'color:{').replace('\n}', '},')\
    .replace('\nscore', 'score').replace('\npixel_fraction', ',pixel_fraction')\
    .replace(' ', '-').replace('\n', '').replace(',-color', '}, color').replace('color:', '{"color":')\
    .replace('red', '"red"').replace('green', '"green"').replace('blue', '"blue"').replace('score', '"score"').replace('pixel_fraction', '"pixel_fraction"')\
    .replace('-', '').replace(',', ';').replace(';"',',"').replace('0"', '0,"').replace(';', ',').replace('},', '}').replace('} {', '},{').replace('}"', '},"')
  
  elif model == 'topics':
    b = string.replace('[', '').replace(']', '}').replace(',','},').replace('/m/', '').replace(' ', '')\
    .replace('mid', '"mid"').replace('description', '"description"').replace('topicality', '"topicality"').replace('score', '"score"').replace('\n', ',').replace(',}', '}')\
    .replace('"mid"', '{"mid"')

  return ast.literal_eval(b)

In [ ]:
#In the same way, we use our previous functions applied to our dataframes, to obtain the color, topic and url data.

def color_measurement(dataframe):

  dataframe['tuple_colors'] = dataframe[['red', 'green', 'blue']].apply(tuple, axis = 1)
  dataframe['colors'] = dataframe.tuple_colors.apply(lambda x: get_colour_name(x))
  dataframe[['type', 'color']] = pd.DataFrame(dataframe['colors'].tolist(), index = dataframe.index)
  
  print(dataframe.columns)
  dataframe.sort_values(['url', 'pixel_fraction'], ascending= [True, False], inplace = True)
  dataframe['section'] = dataframe.groupby('url').cumcount()
  dataframe.reset_index(inplace = True, drop = True)

  return dataframe[(dataframe.section>=0) & (dataframe.section<=2)].reset_index(drop = True)

def topic_measurement(dataframe):
  dataframe.sort_values(['url', 'topicality'], ascending= [True, False], inplace = True)
  dataframe['section'] = dataframe.groupby('url').cumcount()

  return dataframe[(dataframe.section>=0) & (dataframe.section<=2)].reset_index(drop = True)

def text_measurement(dataframe):
  dataframe['description'] = dataframe['description'].str.lower()
  dataframe['description'] = dataframe['description'].map(lambda x: re.sub(r'\n', ' ', x))

  return dataframe

In [ ]:
#Some URLs contain images in compressed formats that are not compatible with cloud vision. We create a function for those images.

def transform_image_is_not_image(response, url, models):

  img = pilmage.open(BytesIO(response.content))

  path = '/content/temporal_file.jpg'

  if 'RGBA' in str(img.mode):
    print('     ', url, 'type' + str(img.mode))
    img = img.convert('RGB')

  img.save(path)

  with io.open(path, 'rb') as image_file:
    content = image_file.read()

  image = vision.Image(content=content)

  if models == 'colors':
    response = client.image_properties(image = image).image_properties_annotation
    dominant_colors = response.dominant_colors

    dictionary = construction_of_json(string = str(dominant_colors.colors), model = models)

    dataframe_colors = pd.concat([DataFrame(list(DataFrame(dictionary)['color'])), DataFrame(dictionary)[['score', 'pixel_fraction']]], axis = 1)

  if models == 'topics':

    res_lebes = client.label_detection(image=image)
    labels = res_lebes.label_annotations
    
    dictionary = construction_of_json(string = str(labels), model = models)

    dataframe_colors = DataFrame.from_dict(dictionary)

  if models == 'texts':
    response = client.text_detection(image = image)
    texts = response.text_annotations

    if len(list(texts)) == 0:
      dataframe_colors = DataFrame({'locale': ['no_text'], 'description':['no_text']})
    else:
      dataframe_colors = DataFrame({'description': [texts[0].description], 'locale': [texts[0].locale]})

  dataframe_colors['url'] = url

  os.remove(path)

  return dataframe_colors.reset_index(drop = True)

Then we create our three main functions, which will consume all of our previous functions. They will perform the extraction of text, color and topics from our images, receiving the dataframe and URL.

In [ ]:
def image_extraction_texts_function(data, image):

  empty = DataFrame()

  for url in list(data):

    image.source.image_uri = url
    response = client.text_detection(image = image)
    texts = response.text_annotations

    if len(list(texts)) == 0:
      response = requests.get(url)
      print('El URL: ', url, 'tiene una respuesta: ', response)

      if '404' not in str(response):
        if '.vmap' not in str(url):
          empty = empty.append(transform_image_is_not_image(response, url, 'texts'))
        else:
          empty = empty.append(DataFrame({'url': [url], 'locale': ['error'], 'description':['error']}))

      else:

        empty = empty.append(DataFrame({'url': [url], 'locale': ['error'], 'description':['error']}))

    elif '.jpg' not in str(url):
      response = requests.get(url)
      print('El URL: ', url, 'tiene una respuesta: ', response)

      empty = empty.append(transform_image_is_not_image(response, url, 'texts'))

    else:

      dataframe_colors = DataFrame({'description': [texts[0].description], 'locale': [texts[0].locale]})
      dataframe_colors['url'] = url

      empty = empty.append(dataframe_colors)

  frame = text_measurement(empty)

  return empty, frame

In [ ]:
def image_extraction_topics_function(data, image):

  empty = DataFrame()

  for url in list(data):

    image.source.image_uri = url
    res_lebes = client.label_detection(image=image)
    labels = res_lebes.label_annotations

    if len(list(labels)) == 0:
      response = requests.get(url)
      print('El URL: ', url, 'tiene una respuesta: ', response)

      if '404' not in str(response):
        if '.vmap' not in str(url):
          empty = empty.append(transform_image_is_not_image(response, url, 'topics'))
        else:
          empty = empty.append(DataFrame({'url': [url], 'mid': [url], 'description':['video'], 'topicality':[0.0], 'score':[0]}))

      else:

        empty = empty.append(DataFrame({'url': [url], 'mid': [url], 'description':['error'], 'topicality':[0.0], 'score':[0]}))

    elif '.jpg' not in str(url):
      response = requests.get(url)
      print('El URL: ', url, 'tiene una respuesta: ', response)

      empty = empty.append(transform_image_is_not_image(response, url, 'topics'))

    else:

      dictionary = construction_of_json(str(labels), 'topics')
  
      dataframe_colors = DataFrame.from_dict(dictionary)

      dataframe_colors['url'] = url

      empty = empty.append(dataframe_colors)

  frame = topic_measurement(empty)

  return empty, frame

In [ ]:
def image_extraction_colors_function(data, image):

  empty = DataFrame()

  for url in list(data):

    image.source.image_uri = url
    response = client.image_properties(image = image).image_properties_annotation
    dominant_colors = response.dominant_colors

    if len(list(dominant_colors.colors)) == 0:
      response = requests.get(url)
      print('El URL: ', url, 'tiene una respuesta: ', response)

      if '404' not in str(response):
        if '.vmap' not in str(url):
          empty = empty.append(transform_image_is_not_image(response, url, 'colors'))
        else:
          pass

      else:

        empty = empty.append(DataFrame({'url': [url], 'pixel_fraction':[132], 'score':[132], 'red':[132], 'green':[132], 'blue':[132]}))

    elif '.jpg' not in str(url):
      response = requests.get(url)
      print('El URL: ', url, 'tiene una respuesta: ', response)

      empty = empty.append(transform_image_is_not_image(response, url, 'colors'))

    else:

      dictionary = construction_of_json(str(dominant_colors.colors), 'colors')
  
      dataframe_colors = pd.concat([DataFrame(list(DataFrame(dictionary)['color'])), DataFrame(dictionary)[['score', 'pixel_fraction']]], axis = 1)

      dataframe_colors['url'] = url

      empty = empty.append(dataframe_colors)
  
  frame = color_measurement(empty)

  return empty, frame

Later we will have to join all the dataframes that we have created, having as primary key our "URL" which will be the only and common element among all the dataframes that we have generated.

In [ ]:
def merge_frames(first_frame, dataframes, model):

  if model == 'colors':
    middle_frame = pd.merge(
      pd.merge(
        pd.merge(first_frame,
          dataframes[dataframes.section == 0][['url','color', 'tuple_colors', 'pixel_fraction']].rename(columns = {'color': 'main_color', 'tuple_colors': 'main_color_id',
                                                                                                                          'pixel_fraction': 'main_pixel_fraction'}),
          how = 'left', on = 'url'),

        dataframes[dataframes.section == 1][['url','color', 'tuple_colors', 'pixel_fraction']].rename(columns = {'color': 'second_color', 'tuple_colors': 'second_color_id',
                                                                                                                          'pixel_fraction': 'second_pixel_fraction'}),
        how = 'left', on = 'url'), 

        dataframes[dataframes.section == 2][['url','color', 'tuple_colors', 'pixel_fraction']].rename(columns = {'color': 'third_color', 'tuple_colors': 'third_color_id',
                                                                                                                        'pixel_fraction': 'third_pixel_fraction'}),
        how = 'left', on = 'url')

    middle_frame['total_pixel_fraction'] = middle_frame.main_pixel_fraction + middle_frame.second_pixel_fraction + middle_frame.third_pixel_fraction

    return middle_frame
  
  elif model == 'topics':
    last_frame = pd.merge(
                          pd.merge(
                            pd.merge(
                              first_frame, 
                              dataframes[(dataframes.section == 0)][['url', 'description', 'topicality']].rename(columns = {'description': 'main_topic', 'topicality': 'main_topicality'}),
                              on = 'url', how = 'left'
                                      ),
                              dataframes[(dataframes.section == 1)][['url', 'description', 'topicality']].rename(columns = {'description': 'second_topic', 'topicality': 'second_topicality'}),
                              on = 'url', how = 'left'
                                  ),
                              dataframes[(dataframes.section == 2)][['url', 'description', 'topicality']].rename(columns = {'description': 'third_topic', 'topicality': 'third_topicality'}),
                              on = 'url', how = 'left'
                            )

    last_frame['accurancy_topic_score'] = (last_frame.main_topicality + last_frame.second_topicality + last_frame.third_topicality) / 3
    last_frame['tuple_topic'] = last_frame[['main_topic', 'second_topic', 'third_topic']].apply(tuple, axis = 1)

    return last_frame

We run our processes and finally send all our synthesized data to bigquery.

In [ ]:
complete_extraction_colors, image_metadata_colors = image_extraction_colors_function(dataframe_dupli.url, image)

In [ ]:
complete_extraction_topics, image_metadata_topics = image_extraction_topics_function(dataframe_dupli.url, image)

In [ ]:
complete_extraction_texts, image_metadata_text = image_extraction_texts_function(dataframe_dupli.url, image)

In [ ]:
semi_last_frame = pd.merge(merge_frames(first_frame = dataframe_dupli[['url', 'source', 'ad_id']], dataframes = image_metadata_colors, model = 'colors'), 
                           merge_frames(first_frame = dataframe_dupli[['url', 'source', 'ad_id']], dataframes = image_metadata_topics, model = 'topics'), 
                           on = ['url', 'source', 'ad_id'],
                           how = 'left')

In [ ]:
final_frame = pd.merge(semi_last_frame, image_metadata_text, on = 'url', how = 'left')

In [ ]:
final_frame.to_gbq(destination_table = 'clients_table.main_categorical_values', 
                   project_id = 'omg-latam-prd-adh-datahouse-cl', 
                   if_exists = 'replace') 